# Import libraries, set options, connect to DB

In [1]:
# Configuration code for datawrangling
import pandas as pd
import os
import numpy as np
from datetime import datetime
from geocode import geocode
import mapToPoly
from mapToPoly import mapToPoly
pd.set_option('display.max_row', 30000)
import csv

# Configuration code in order to connect to the database
from sqlalchemy import create_engine, exists
from sqlalchemy.orm import sessionmaker
from database_setup import Itenerary, Base

passWord = os.environ['my_password']
DATABASE_URI = 'postgres://maxcarey:' + passWord + '@totago.cqfm37jhmjmk.ap-southeast-2.rds.amazonaws.com:5432/totago'
engine = create_engine(DATABASE_URI)

#engine = create_engine('sqlite:///totagoData.db')

# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)

session = DBSession()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


True
True


# Read in data as pandas data frame, selecting only certain fields

In [2]:
fields = ['distinct_id', 'numItinerariesReturned', 'departureDate', 'startFromLocation', 'selectedDestination_id', 'selectedDestination_name', 'time']

In [3]:
df = pd.read_csv('generated_itineraries.csv', usecols = fields)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Wrange field: destinationIDs

In [4]:
# Replace all of the NAs for destinationIDs with 0
df.selectedDestination_id.fillna(0, inplace = True)

# Remove the 2 cases where the string says null
# Great tutorial here: https://www.youtube.com/watch?v=2AFGPdNn4FM
df = df[df.selectedDestination_id != 'null']

# Convert destinationIDs column to an integer value
df['selectedDestination_id'] = df.selectedDestination_id.astype(int)



# Wrangle field: numItenerariesReturned

In [5]:
# Replace all of the NAs for numItinerariesReturned with 1
df.numItinerariesReturned.fillna(1, inplace = True)

# Convert from float to integer
df['numItinerariesReturned'] = df.numItinerariesReturned.astype(int)

# Select, only observatiosn where this field is greater than 0 (now that the NAs are gone)


# Wrangle Field: Destination Name

In [6]:
#Replace all of the NAs in
df.selectedDestination_name.fillna("", inplace = True)

print("Number of rows before departure date: ")
print(len(df))


Number of rows before departure date: 
28138


# Wrangle Field: departureDate

In [7]:
#Convert destinationIDs column to an integer value
# It looks like there were some complex rows being held in here before, I thought that when df.dtypes returned object that
# meant string but apprently not
df['departureDate'] = df.departureDate.astype(str)

print("number of rows before removal of anamoulous departureDate cases")
print(len(df))


# IT looks like there are some cases where this field is blank, says nan, is in format 24503, or in format "masked" 
# We need to remove these cases from the data frame
# I can see that some blank rows are still printed out.
df = df[df.departureDate != '']
df = df[df.departureDate != 'nan']
df = df[df.departureDate != '24503']
df = df[df.departureDate != '[masked]']


print("number of rows after removal of anamoulous departureDate cases")
print(len(df))

# Create a function extractDate that extracts the first ten characters of an input string
def extractDate(dateString):
    extractedDate = dateString[0:10]
    if len(extractedDate) < 10:
        print(extractedDate)
    return extractedDate

''' Code to test if the extractDate function works

# Apply this function to create  a new column
df['departureDateFixed'] = df.departureDate.apply(extractDate)

cols = ['distinct_id', 'departureDate', 'departureDateFixed', 'numItinerariesReturned', 'selectedDestination_id', 'selectedDestination_name', 'startFromLocation']

df = df[cols]
'''

# Override departure date extracting all of the null time stamps
df['departureDate'] = df.departureDate.apply(extractDate)


# Convert departure date into a time object in pandas
#See here: https://stackoverflow.com/questions/26763344/convert-pandas-column-to-datetime
# Though this actually might not need to be done
#df['departureDate'] = df.departureDate.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))




#df['departureDate'] = datetime.strptime(df['departureDate'], '%Y-%m-%-d')  
#df['departureDate'] = pd.to_datetime(df['departureDate'], format = '%Y-%m-%-d')



number of rows before removal of anamoulous departureDate cases
28138
number of rows after removal of anamoulous departureDate cases
28130


# Wrangle Field: distinctID

In [8]:
#It turns out distinc_id correpsonds to a user

# Therefore, create a coloumn that combines the unix time stamp with distinct_id so that we have a primary key for database
df["primary_key"] = df["distinct_id"] + "-" + df["time"].map(str)


vc = df.primary_key.value_counts()
print(vc[vc > 1])


unique_keys = df.primary_key.unique()
#print(len(unique_keys))

#df.head(n = len(df))

2578f3c5-2bd8-4d7e-b05e-d82eb4e24632-1448989343                             3
15ae497624281a-06e6a097b99f59-5e4f2b18-ff000-15ae4976243a16-1489889250      3
16017f5f592536-094b655d7b8ce4-7636321b-4a640-16017f5f5932bd-1512207331      3
68f3348e-32ff-4756-a6b9-fbf722d5bf76-1460184823                             3
156c0427d8411b-089cb3f5e0f3f-1c114a5c-13c680-156c0427d85c5-1472086519       3
9a942f21-ad25-4129-b760-46bcec9e631d-1448989392                             2
2578f3c5-2bd8-4d7e-b05e-d82eb4e24632-1448989357                             2
9a942f21-ad25-4129-b760-46bcec9e631d-1448989377                             2
2578f3c5-2bd8-4d7e-b05e-d82eb4e24632-1448989361                             2
9a942f21-ad25-4129-b760-46bcec9e631d-1448989346                             2
2578f3c5-2bd8-4d7e-b05e-d82eb4e24632-1448989332                             2
1618f2648b084b-06c2b14d540606-32657403-1aeaa0-1618f2648b147f-1518645763     2
c7ca4b64-84f0-46dd-b8e1-00530a09f7d9-1443084797                 

# Create a subset of the datle with sample method to test geocode and database entry logic

In [9]:
#Out put the entire database
#df.head(len(df))

len(df)

28130

In [10]:
#Create a random sample of the database, these entries will be added to the database in the next section
sampleDf = df.sample(2000)

# Output this random sample
sampleDf.head(len(sampleDf))    

,departureDate,distinct_id,numItinerariesReturned,selectedDestination_id,selectedDestination_name,startFromLocation,time,primary_key
14566,2017-07-29,155d1cdffa0319-04d765e01-26792551-3d10d-155d1c...,0,130,,"565 Smithe Street, Vancouver, BC, Canada",1501317439,155d1cdffa0319-04d765e01-26792551-3d10d-155d1c...
24768,2018-08-01,164f3c19b772b4-0861a5ad39c4658-31126a5d-2c600-...,1,46,Norvan Falls,"New Westminster Station, New Westminster, BC, ...",1533072925,164f3c19b772b4-0861a5ad39c4658-31126a5d-2c600-...
2866,2016-07-23,156149b602a29b-047a7d41c-684a317b-3d10d-156149...,1,146,,"Hornby Street, Vancouver, BC, Canada",1469229603,156149b602a29b-047a7d41c-684a317b-3d10d-156149...
21517,2018-05-12,1633b282f99442-08a1deebd-79783863-13c680-1633b...,1,118,Buntzen Lake,"989 Richards Street, Vancouver, BC, Canada",1526116819,1633b282f99442-08a1deebd-79783863-13c680-1633b...
5441,2016-09-08,1570be9aadb3c0-0faaf1dd8b942d-78611f35-1fa400-...,1,146,,"12040 68 Ave, Surrey, BC, Canada",1473348477,1570be9aadb3c0-0faaf1dd8b942d-78611f35-1fa400-...
15528,2017-08-19,15dfb5dcac5abf-05cae4ce969822-3a3e5c06-384000-...,1,110,Admiralty Point,"821 Cambie Street, Vancouver, BC, Canada",1503135618,15dfb5dcac5abf-05cae4ce969822-3a3e5c06-384000-...
4773,2016-08-26,156ca92e29e28-0e214ad69cc2ff-4049042b-100200-1...,1,140,,"Vancouver, BC, Canada",1472252240,156ca92e29e28-0e214ad69cc2ff-4049042b-100200-1...
8451,2017-02-13,15a35183cfccf-040fb85e9b262d-57e1b3c-232800-15...,1,0,,"620 West Pender Street, Vancouver, BC, Canada",1486923291,15a35183cfccf-040fb85e9b262d-57e1b3c-232800-15...
16931,2017-09-24,15eb46f952319-01ee65e8d01718-6b713c72-3d10d-15...,1,561,The Hollywood Sign (Mount Lee),"Griffith Observatory, Los Angeles, CA, United ...",1506256734,15eb46f952319-01ee65e8d01718-6b713c72-3d10d-15...
5830,2016-09-17,1573666a48469-00ba6e63df3ba6-414f0020-fa000-15...,1,149,,"10082 148 Street, Surrey, BC, Canada",1474061320,1573666a48469-00ba6e63df3ba6-414f0020-fa000-15...


## Read in the destination data to allow the possibility to pull the correct names

 


In [11]:
f = open("destinations_mapping_Jul-30-18.csv")

reader = csv.reader(f)


destinations = {}


# The index at the end of the for loop just skips the first row which is the header in the csv file
next(reader)
for row in reader:
    destinations[row[0]] = {'name':row[1]}

print(destinations)

{'168': {'name': 'Mammoth Pass - Crater Meadow Trail'}, '178': {'name': 'Sport Climbing at Horseshoe Slabs'}, '181': {'name': 'Hazel Wolf Wetlands Loop'}, '183': {'name': 'Redwood Grove Loop Trail'}, '184': {'name': 'Birdwatching at Fowlsheugh Reserve'}, '12': {'name': 'Point Defiance Loop (OLD)'}, '112': {'name': 'Baden Powell Lynn Canyon to Grouse'}, '175': {'name': 'Rainbow Falls Trail'}, '174': {'name': 'Walking the Town Loop to Sherwins Vista'}, '20': {'name': 'The Lions Binkert Trail'}, '27': {'name': 'Tumamoc Hill'}, '8': {'name': 'Proximity Alert Test'}, '169': {'name': 'McLeod Lake Spur'}, '165': {'name': 'Siskiyou Mountain (White Rabbit Trail)'}, '22': {'name': 'Lake Blanca'}, '11': {'name': 'Carkeek Park'}, '162': {'name': 'Rouge Park Traverse'}, '172': {'name': 'Walking the Lakes Basin Path at Twin Lakes'}, '16': {'name': 'Golden Gardens to Carkeek Park Beach Walk'}, '111': {'name': 'Baden Powell Deep Cove to Lynn Canyon'}, '278': {'name': 'Tujunga Wash Path'}, '13': {'name

# Loop through the rows in the dataframe, geocode, add entry to database

In [ ]:
# Loop through the subsetted pandas data frame

# Uncomment the code below to loop through the the sample data frame
# for index, row in sampleDf.iterrows():

for index, row in sampleDf.iterrows():
  

    # Pull out the primary key into a variable
    testKey = row["primary_key"]
    
    # Check to see if that distinctID is in the data base
    # See this post: https://stackoverflow.com/questions/6587879/how-to-elegantly-check-the-existence-of-an-object-instance-variable-and-simultan?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
    entryExists = session.query(exists().where(Itenerary.distinctkey==testKey)).scalar()

    # If the entry is not in the database
    if not entryExists:
    
        # Get the string to be geocoded
        locationToGeocode = row["startFromLocation"]

        # Try to run the geocode function that returns a dictionary of information
        try:
            geocodeInfo = geocode(locationToGeocode)
            # If geocoding works, set valid to tre
            valid = True

        # If the geocode function doesn't work set valid to false
        except:
            valid = False 

        # If valid is true create a database entry with information from the dataframe, and the returned geocode informaiton
        if valid:
            
            # Sometimes, such as when a generic city is sent to the geocode() function a geometric center
            # is returned, this means there is no postal code
            
            # In this case, we can set the postalCode to One
            if not 'postalCode' in geocodeInfo:
                geocodeInfo['postalCode'] = "none"

            
            # Mapp the gps coordinates returned to the zip code polygons
            zipCodeMapped = mapToPoly(geocodeInfo['lat'], geocodeInfo['lng'], 'postal')
            
            barrioMapped = mapToPoly(geocodeInfo['lat'], geocodeInfo['lng'], 'barrio')
            
            ## Get selected Destination Names
            # Pull the selected destination name
            selectedDestinationName = row["selectedDestination_name"]
            
            if not selectedDestinationName:
                
                key = str(row["selectedDestination_id"])
                
                if key in destinations:
            
                    # Pull the data out from the dictionary that was created in the cell above
                    newName = destinations[str(row["selectedDestination_id"])]['name']
        
                    # Add the new name to the new row
                    selectedDestinationName  = newName
            
                # In the case that there is destination that corresponds mark
                else:
                
                    # TODO: CONSIDER CHANGING THE NAME OF THIS TO SOMETHING ELSE
                    selectedDestinationName = "DELETED"
                    # And overwrite valid to false at this point because there is no destination
                    valid = False
                
            databaseEntry = Itenerary(distinctkey=row["primary_key"],
                                      numberitinerariesreturned=row["numItinerariesReturned"],
                                      selecteddestination_id=row["selectedDestination_id"],
                                      selecteddestination_name=selectedDestinationName,
                                      startfromlocation=row["startFromLocation"],
                                      departuredate=row["departureDate"],
                                      # Get data from python dictionary returned from geocode() function
                                      formatted_address=geocodeInfo['formatted_address'],
                                      lat=geocodeInfo['lat'],
                                      lng=geocodeInfo['lng'],
                                      postalcode=geocodeInfo['postalCode'],
                                      postalcodemapped=zipCodeMapped,
                                      barriomapped=barrioMapped,
                                      valid=valid)
        # If valid is false, just fill in the information that we have from the pandas data frame
        else:
            databaseEntry = Itenerary(distinctkey=row["primary_key"],
                                      numberitinerariesreturned=row["numItinerariesReturned"],
                                      selecteddestination_id=row["selectedDestination_id"],
                                      selecteddestination_name=row["selectedDestination_name"],
                                      startfromlocation=row["startFromLocation"],
                                      departuredate=row["departureDate"],
                                      valid=valid)

        # Add the the information to a database.    
        session.add(databaseEntry)
        session.commit()
    
    else:
        print("Entry already inside database")

V6B
CBD
V3M
V6Z
CBD
V6B
CBD
V3W
V6B
CBD
V6C
CBD
V6B
CBD
90027
115295
V3R
V3K
V6K
KITS
V6K
KITS
V6K
KITS
V6C
CBD
90291
21056
V7J
V6S
DS
98164
271849
V6B
CBD
V6E
WE
V6T
V6Z
CBD
V5C
V5X
SUN
V5Y
RP
V6T
WPG
V6C
CBD
V7M
V6B
CBD
V6P
MARP
V5V
RP
V5X
MARP
98115
250780
V7T
V6P
MARP
V6A
CBD
V5N
KC
V5X
SUN
V5N
GW
V6E
WE
V6Z
CBD
V6T
98115
251709
V6N
DS
V6G
WE
V6Z
CBD
98105
252248
V6K
KITS
90015
268118
V7Y
CBD
V6B
CBD
98164
271849
V3J
V5W
SUN
V6C
CBD
V6E
WE
V5N
GW
V6T
V5V
RP
V6Y
V6C
CBD
98101
271849
V6T
V6C
CBD
V6C
CBD
V5R
RC
V6G
WE
V6Z
CBD
V6K
KITS
V6G
WE
V6Z
CBD
98164
271849
90095
118920
98107
250017
90028
32059
V6K
KITS
V6Z
CBD
Entry already inside database
Entry already inside database
98107
271961
V6Z
CBD
98164
271849
V6B
CBD
98122
271869
V5N
KC
V7Y
CBD
91205
275396
V6C
CBD
V5Z
FAIR
V7M
V7M
V5N
GW
V5E
V3M
98004
271850
V6T
V5L
GW
V6B
CBD
V6E
WE
98109
272022
V6T
V5V
RP
V6B
CBD
V6T
V6Z
CBD
V6T
98104
271869
98109
271987
V3J
V6P
MARP
V3K
V7B
V7Y
CBD
V0N
V6E
WE
V6C
CBD
91405
268548
V5N
GW
V3J
V6E
WE


10007
270951
V4B
V6G
WE
V6G
CBD
V6R
KITS
V6C
CBD
V6G
WE
V5R
RC
V3X
V7C
98109
271987
V5W
RP
V5S
KIL
V7T
V4B
V6E
WE
V6T
V6Z
CBD
V7Y
CBD
V6R
KITS
98133
250801
V5C
V6B
CBD
98106
344034
98660
271863
Entry already inside database
V1M
V6G
CBD
V5Z
FAIR
98121
271808
98003
251017
V5H
V6H
FAIR
V5Z
SC
V7Y
CBD
V6E
WE
Entry already inside database
98164
271849
V5A
V6G
WE
V6B
CBD
V6E
CBD
V7Y
CBD
V6S
98105
272001
V6E
WE
98105
272001
V6B
CBD
V3J
98105
252248
V5B
V5E
98102
250206
V6B
CBD
V6C
CBD
V5G
V6B
CBD
V6C
CBD
V6R
WPG
V5C
V6B
CBD
98126
344030
V6B
CBD
98105
272001
V7K
V6E
WE
V6T
V5R
RC
V5R
RC
V4A
V6J
SHAU
V6P
MARP
98058
V6B
CBD
98164
271849
V6J
FAIR
V5L
GW
V6C
CBD
V6Y
90503
416338
V5H
V6T
V5X
SUN
V6E
WE
V6E
WE
V6T
98103
250692
V6T
V6G
V5J
V3J
V5V
RP
Entry already inside database
V5R
RC
V6C
CBD
V5K
HS
V6R
WPG
V6H
FAIR
98118
343994
V6Y
V6S
Entry already inside database
98105
272001
V5N
GW
Entry already inside database
V7R
V6R
WPG
V7M
91331
40215
V5R
RC
V3C
V5K
HS
V7W
V7M
V6E
WE
V5M
RC
V6P
MARP
V7M
V5T